# Problem 1 - Learning Rate, Batch Size, FashionMNIST

Recall cyclical learning rate policy discussed in Lecture 4. The learning rate changes in cyclical manner between lrmin and lrmax, which are hyperparameters that need to be specified. For this problem you first need to read carefully the article referenced below as you will be making use of the code there (in Keras) and modifying it as needed. For those who want to work in Pytorch there are open source implementations of this policy available which you can easily search for and build over them. You will work with FashionMNIST dataset and LeNet-5.

References:
1. Leslie N. Smith Cyclical Learning Rates for Training Neural Networks. Available at https://arxiv.org/abs/1506.01186.
2. Keras implementation of cyclical learning rate policy. Available at https://www.pyimagesearch.com/2019/08/05/keras-
learning-rate-finder/.


1. Fix batch size to 64 and start with 10 candidate learning rates between 10−9 and 101 and train your model for 5 epochs for each learning rate. Plot the training loss as a function of learning rate. You should see a curve like Figure 3 in reference below. From that figure identify the values of lrmin and lrmax. 

In [1]:
! conda install ipykernel --name Python3
! python -m ipykernel install
! pip3 install cv2


EnvironmentLocationNotFound: Not a conda environment: /Users/aragaom/opt/anaconda3/envs/Python3

Installed kernelspec python3 in /usr/local/share/jupyter/kernels/python3
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
You should consider upgrading via the '/Users/aragaom/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [5]:
# import the necessary packages
import os
# initialize the list of class label names
CLASSES = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]
# define the minimum learning rate, maximum learning rate, batch size,
# step size, CLR method, and number of epochs
MIN_LR = 1e-10
MAX_LR = 1e1
BATCH_SIZE = 64
STEP_SIZE = 5
CLR_METHOD = "triangular"
NUM_EPOCHS = 50
# define the path to the output learning rate finder plot, training
# history plot and cyclical learning rate plot
LRFIND_PLOT_PATH = os.path.sep.join(["output", "lrfind_plot.png"])
TRAINING_PLOT_PATH = os.path.sep.join(["output", "training_plot.png"])
CLR_PLOT_PATH = os.path.sep.join(["output", "clr_plot.png"])

In [2]:
from tensorflow.keras import datasets, layers, models, losses

def create_model():
  model = models.Sequential()

  model.add(layers.Conv2D(6, 5, activation='tanh', input_shape=trainX.shape[1:]))
  model.add(layers.AveragePooling2D(2))


  model.add(layers.Conv2D(16, 5, activation='tanh'))
  model.add(layers.AveragePooling2D(2))

  model.add(layers.Conv2D(120, 5, activation='tanh'))

  model.add(layers.Flatten()) # dense layer is a linear layer and we flatten the input before putting it there.
  model.add(layers.Dense(84, activation='tanh'))

  model.add(layers.Dense(10, activation='softmax'))

  return model

In [3]:
# import tensorflow as tf
# def load_data_mnist_tf(batch_size, resize=None):   
    
#     # load dataset
#     mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()

#     # normalisation and cast as Int datatype
#     process = lambda X, y: (tf.expand_dims(X, axis=3) / 255,tf.cast(y, dtype='int32')) 
#     # the pixel values must be personalized, so each feature has the same affect ont he output 

#     # resize images if resize is not None
#     resize_fn = lambda X, y: (tf.image.resize_with_pad(X, resize, resize) if resize else X, y)
#     # resizing of the image fucntion ?? 


#     # load train and test batches
#     train_iter = tf.data.Dataset.from_tensor_slices(process(*mnist_train)).batch(batch_size).shuffle(len(mnist_train[0])).map(resize_fn)
#     test_iter = tf.data.Dataset.from_tensor_slices(process(*mnist_test)).batch(batch_size).map(resize_fn)
    
#     return (train_iter, test_iter)

In [33]:
# set the matplotlib backend so figures can be saved in the background
# import matplotlib
# matplotlib.use("Agg")
# import the necessary packages
from learningratefinder import LearningRateFinder
from clr_callback import CyclicLR
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import numpy as np
import argparse
# import cv2
import sys

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-f", "--lr-find", type=int, default=0,
	help="whether or not to find optimal learning rate")
args, unknown = ap.parse_known_args()

resize_fn = lambda X, y: (tf.image.resize_with_pad(X, resize, resize) if resize else X, y)
# load the training and testing data
print("[INFO] loading Fashion MNIST data...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
# Fashion MNIST images are 28x28 but the network we will be training
# is expecting 32x32 images
# trainX = np.array([tf.image.resize(x, [32,32]) for x in trainX])
# testX = np.array([tf.image.resize(x [32,32]) for x in testX])
# scale the pixel intensities to the range [0, 1]
trainX = tf.pad(trainX, [[0, 0], [2,2], [2,2]])/255
testX = tf.pad(testX, [[0, 0], [2,2], [2,2]])/255

trainX = tf.expand_dims(trainX, axis=3, name=None)
testX = tf.expand_dims(testX, axis=3, name=None)


# reshape the data matrices to include a channel dimension (required
# for training)
# trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
# testX = testX.reshape((testX.shape[0], 28, 28, 1))
# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)
# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
	height_shift_range=0.1, horizontal_flip=True,
	fill_mode="nearest")

[INFO] loading Fashion MNIST data...


In [7]:
print(args)
if args.lr_find == 0:
    	# initialize the learning rate finder and then train with learning
	# rates ranging from 1e-10 to 1e+1
	print("[INFO] finding learning rate...")
	lrf = LearningRateFinder(model)
	lrf.find(
		aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
		1e-10, 1e+1,
		stepsPerEpoch=np.ceil((len(trainX) / float(BATCH_SIZE))),epochs=50,
		batchSize=BATCH_SIZE)
	# plot the loss for the various learning rates and save the
	# resulting plot to disk
	lrf.plot_loss()
	plt.savefig("learn_rate_finder.png")
	# gracefully exit the script so we can adjust our learning rates
	# in the config and then train the network for our full set of
	# epochs
	print("[INFO] learning rate finder complete")
	print("[INFO] examine plot and adjust learning rates before training")
	sys.exit(0)

Namespace(lr_find=0)
[INFO] finding learning rate...
Epoch 1/50
938/938 [==============================] - 20s 21ms/step - loss: 2.3042 - accuracy: 0.0889
Epoch 2/50
938/938 [==============================] - 22s 24ms/step - loss: 2.3040 - accuracy: 0.0904
Epoch 3/50
938/938 [==============================] - 26s 27ms/step - loss: 2.3041 - accuracy: 0.0891
Epoch 4/50
938/938 [==============================] - 22s 23ms/step - loss: 2.3041 - accuracy: 0.0884
Epoch 5/50
938/938 [==============================] - 21s 23ms/step - loss: 2.3041 - accuracy: 0.0881
Epoch 6/50
938/938 [==============================] - 24s 26ms/step - loss: 2.3040 - accuracy: 0.0893
Epoch 7/50
938/938 [==============================] - 21s 22ms/step - loss: 2.3039 - accuracy: 0.0903
Epoch 8/50
938/938 [==============================] - 23s 25ms/step - loss: 2.3040 - accuracy: 0.0888
Epoch 9/50
938/938 [==============================] - 21s 23ms/step - loss: 2.3042 - accuracy: 0.0895
Epoch 10/50
938/938 [========

SystemExit: 0

/Users/aragaom/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


***See figure learn_rate_finder.png***

2. Use the cyclical learning rate policy (with exponential decay) and train your network using batch size 64 and lrmin and lrmax values obtained in part 1. Plot train/validation loss and accuracy curve (similar to Figure 4 in reference).

***Answer:***

if you look at the learn_rate_finder.png file, you will notice that the loss starts to decrease with the learn rate 1e-5 and then spikes up after the learn rate 1e-2.

In [44]:
MIN_LR = 1e-5
MAX_LR = 1e-2

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(learning_rate=MIN_LR, momentum=0.9)
model = create_model()
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])


[INFO] compiling model...


In [45]:
stepSize = STEP_SIZE * (trainX.shape[0] // BATCH_SIZE)
clr = CyclicLR(
	mode=CLR_METHOD,
	base_lr=MIN_LR,
	max_lr=MAX_LR,
	step_size=stepSize)
# train the network
print("[INFO] training network...")
H = model.fit(
	x=aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
	validation_data=(testX, testY),
	steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
	epochs=NUM_EPOCHS,
	callbacks=[clr],
	verbose=1)
# evaluate the network and show a classification report
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BATCH_SIZE)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=CLASSES))

[INFO] training network...
Epoch 1/50
937/937 [==============================] - 22s 22ms/step - loss: 1.6415 - accuracy: 0.4202 - val_loss: 0.9228 - val_accuracy: 0.6700
Epoch 2/50
937/937 [==============================] - 20s 22ms/step - loss: 0.8634 - accuracy: 0.6818 - val_loss: 0.7052 - val_accuracy: 0.7247
Epoch 3/50
937/937 [==============================] - 20s 21ms/step - loss: 0.7005 - accuracy: 0.7338 - val_loss: 0.5840 - val_accuracy: 0.7802
Epoch 4/50
937/937 [==============================] - 24s 26ms/step - loss: 0.6128 - accuracy: 0.7646 - val_loss: 0.5261 - val_accuracy: 0.7985
Epoch 5/50
937/937 [==============================] - 24s 25ms/step - loss: 0.5496 - accuracy: 0.7914 - val_loss: 0.4930 - val_accuracy: 0.8094
Epoch 6/50
937/937 [==============================] - 24s 26ms/step - loss: 0.4912 - accuracy: 0.8147 - val_loss: 0.4462 - val_accuracy: 0.8315
Epoch 7/50
937/937 [==============================] - 23s 25ms/step - loss: 0.4496 - accuracy: 0.8324 - val_l

In [46]:
N = np.arange(0, NUM_EPOCHS)
# plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("training_plot.png")
# plot the learning rate history
N = np.arange(0, len(clr.history["lr"]))
plt.figure()
plt.plot(N, clr.history["lr"])
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Training Iterations")
plt.ylabel("Learning Rate")
plt.savefig("crl_plot.png")

3. We want to test if increasing batch size for a fixed learning rate has the same effect as decreasing learning rate for a fixed batch size. Fix learning rate to lrmax and train your network starting with batch size 32 and incrementally going upto 4096 (in increments of a factor of 2; like 32, 64...). You can choose a step size (in terms of number of epochs) to increment the batch size. Plot the training loss vs. log2(batch size). Is the generalization of your final model similar or different than cyclical learning rate policy?


***Answer:***

Very very similar, but the biggest trade-off is that by increasing the batches I reduce training time by almost 25% of the time spent with the cyclical learning rate, possibly, because as we increase the batch, we need fewer iterations at each epoch.

In [47]:
batch = 32

print("[INFO] compiling model...")
opt = SGD(learning_rate=MAX_LR, momentum=0.9)
model = create_model()
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

loss = []
batches = []

while batch <=4096:
    print("[INFO] training network...")
    H_b = model.fit(
	x=aug.flow(trainX, trainY, batch_size=batch),
	validation_data=(testX, testY),
	steps_per_epoch=trainX.shape[0] // batch,
	epochs=5,
	# callbacks=[clr],
	verbose=1)
    # evaluate the network and show a classification report
    print("[INFO] evaluating network...")
    predictions = model.predict(x=testX, batch_size=batch)
    print(classification_report(testY.argmax(axis=1),
        predictions.argmax(axis=1), target_names=CLASSES))
    
    loss.append(H_b.history["loss"][-1])
    batches.append(batch)
    
    # updating the batch size
    batch*=2
    


[INFO] compiling model...
[INFO] training network...
Epoch 1/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.7799 - accuracy: 0.7033 - val_loss: 0.5456 - val_accuracy: 0.7917
Epoch 2/5
1875/1875 [==============================] - 23s 12ms/step - loss: 0.5524 - accuracy: 0.7894 - val_loss: 0.5259 - val_accuracy: 0.8023
Epoch 3/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.4855 - accuracy: 0.8187 - val_loss: 0.4129 - val_accuracy: 0.8428
Epoch 4/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.4445 - accuracy: 0.8338 - val_loss: 0.4283 - val_accuracy: 0.8403
Epoch 5/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.4210 - accuracy: 0.8427 - val_loss: 0.3864 - val_accuracy: 0.8526
[INFO] evaluating network...
313/313 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

         top       0.80      0.81      0.80      1000
     trouser       0.98      

In [42]:
plt.figure()
plt.plot(batches, loss)
plt.title("Batches vs loss")
plt.xlabel("Batch Size")
plt.ylabel("Training Loss")
plt.savefig("batches_loss.png")